# BERT masked model on the full PE dataset
### try to check if it works

In [1]:
!pip install transformers
!pip install ipywidgets
!pip install IProgress
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import transformers
import torch

In [3]:
from transformers import BertTokenizer, BertForMaskedLM

In [4]:
import pandas as pd

### make the train, test dataframe

In [5]:
df = pd.read_pickle("/notebooks/Prompting/dataset/pe_dataset_w_prompts_1_pickle")

In [6]:
df_sans_mc = df[df.label_ComponentType != 'MajorClaim']

In [7]:
df_sans_mc

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,nr_preceeding_comps_in_para,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_1
2,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,TRAIN,...,0,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
3,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,TRAIN,...,1,2,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, PRON, VERB, ADP, NO...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
4,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,TRAIN,...,2,1,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, DET, NOUN, ADP, NOUN...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
5,essay001,T6,Premise 1088 1191,All of these skills help them to get on well w...,Premise,Premise,[],Support,NotLinked,TRAIN,...,3,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADP, DET, NOUN, VERB...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
6,essay001,T7,Claim 1332 1376,competition makes the society more effective,Claim,Claim,[],Attack,Linked,TRAIN,...,1,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.75,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: NOUN, VERB, DET, NOUN, AD...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5968,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,TRAIN,...,4,3,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: ADV, PRON, VERB, VERB, AD...",Topic: Children should studying hard or playin...,Component: Topic: Children should studying har...
5969,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,TRAIN,...,5,2,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, VERB, VERB, NOUN, VE...",Topic: Children should studying hard or playin...,Component: Topic: Children should studying har...
5970,essay402,T13,Premise 1393 1436,they can contribute positively to community,Premise,Premise,[],Support,Linked,TRAIN,...,6,1,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: PRON, VERB, VERB, ADV, AD...",Topic: Children should studying hard or playin...,Component: Topic: Children should studying har...
5971,e

In [8]:
df_sans_mc = df_sans_mc.reset_index(drop=True)

In [9]:
df_sans_mc

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,nr_preceeding_comps_in_para,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_1
0,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,TRAIN,...,0,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
1,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,TRAIN,...,1,2,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, PRON, VERB, ADP, NO...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
2,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,TRAIN,...,2,1,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, DET, NOUN, ADP, NOUN...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
3,essay001,T6,Premise 1088 1191,All of these skills help them to get on well w...,Premise,Premise,[],Support,NotLinked,TRAIN,...,3,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADP, DET, NOUN, VERB...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
4,essay001,T7,Claim 1332 1376,competition makes the society more effective,Claim,Claim,[],Attack,Linked,TRAIN,...,1,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.75,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: NOUN, VERB, DET, NOUN, AD...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5234,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,TRAIN,...,4,3,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: ADV, PRON, VERB, VERB, AD...",Topic: Children should studying hard or playin...,Component: Topic: Children should studying har...
5235,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,TRAIN,...,5,2,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, VERB, VERB, NOUN, VE...",Topic: Children should studying hard or playin...,Component: Topic: Children should studying har...
5236,essay402,T13,Premise 1393 1436,they can contribute positively to community,Premise,Premise,[],Support,Linked,TRAIN,...,6,1,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: PRON, VERB, VERB, ADV, AD...",Topic: Children should studying hard or playin...,Component: Topic: Children should studying har...
5237,e

In [10]:
df_sans_mc.label_ComponentType.value_counts()

Premise    3763
Claim      1476
Name: label_ComponentType, dtype: int64

In [11]:
# ok

In [12]:
df_sans_mc.split.value_counts()

TRAIN    4132
TEST     1107
Name: split, dtype: int64

### get the tokenizer

In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

## make train, test datasets

### train dataset

In [14]:
df_train = df_sans_mc[df_sans_mc.split == 'TRAIN']

In [15]:
df_train = df_train.reset_index(drop=True)

In [16]:
df_train

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,nr_preceeding_comps_in_para,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_1
0,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,TRAIN,...,0,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
1,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,TRAIN,...,1,2,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, PRON, VERB, ADP, NO...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
2,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,TRAIN,...,2,1,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, DET, NOUN, ADP, NOUN...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
3,essay001,T6,Premise 1088 1191,All of these skills help them to get on well w...,Premise,Premise,[],Support,NotLinked,TRAIN,...,3,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADP, DET, NOUN, VERB...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
4,essay001,T7,Claim 1332 1376,competition makes the society more effective,Claim,Claim,[],Attack,Linked,TRAIN,...,1,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.75,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: NOUN, VERB, DET, NOUN, AD...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4127,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,TRAIN,...,4,3,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: ADV, PRON, VERB, VERB, AD...",Topic: Children should studying hard or playin...,Component: Topic: Children should studying har...
4128,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,TRAIN,...,5,2,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, VERB, VERB, NOUN, VE...",Topic: Children should studying hard or playin...,Component: Topic: Children should studying har...
4129,essay402,T13,Premise 1393 1436,they can contribute positively to community,Premise,Premise,[],Support,Linked,TRAIN,...,6,1,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: PRON, VERB, VERB, ADV, AD...",Topic: Children should studying hard or playin...,Component: Topic: Children should studying har...
4130,e

In [17]:
df_train.split.value_counts()

TRAIN    4132
Name: split, dtype: int64

In [18]:
prompted_texts_l = df_train['prompted_representation_1'][:].tolist()

In [19]:
len(prompted_texts_l)

4132

In [20]:
inputs_train = tokenizer(prompted_texts_l, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [21]:
inputs_train

{'input_ids': tensor([[ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        ...,
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [22]:
inputs_train['input_ids'].shape

torch.Size([4132, 512])

In [23]:
# for training we will need labels

In [24]:
inputs_train['labels'] = inputs_train.input_ids.detach().clone()

In [25]:
inputs_train

{'input_ids': tensor([[ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        ...,
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        ...,
      

In [26]:
inputs_train['labels'].shape

torch.Size([4132, 512])

In [27]:
# find where the last 102 is

In [28]:
(inputs_train['input_ids'][0] == 102).nonzero(as_tuple=True)[0].item()

145

In [29]:
tokenizer.decode(4366)

'c l a i m'

In [30]:
tokenizer.decode(18458)

'p r e m i s e'

In [31]:
# claim = 4366, premise = 18458

In [32]:
(inputs_train['input_ids'][1] == 102).nonzero(as_tuple=True)[0].item()

171

In [33]:
inputs_train['input_ids'][1][169]

tensor(18458)

In [34]:
# confirm that all at [len-2] indices are either claim or premise

In [35]:
cls_indices_list = []

for i in range(len(prompted_texts_l)):
    cls_idx = (inputs_train['input_ids'][i] == 102).nonzero(as_tuple=True)[0].item()
    cls_indices_list.append(cls_idx)

In [36]:
cls_indices_list

[145,
 171,
 220,
 150,
 101,
 130,
 152,
 288,
 114,
 143,
 113,
 188,
 139,
 149,
 124,
 149,
 119,
 129,
 176,
 142,
 135,
 105,
 140,
 179,
 151,
 131,
 119,
 208,
 141,
 125,
 131,
 221,
 207,
 161,
 153,
 136,
 119,
 124,
 157,
 127,
 136,
 198,
 180,
 170,
 148,
 170,
 143,
 170,
 165,
 127,
 104,
 92,
 111,
 133,
 155,
 109,
 131,
 100,
 124,
 98,
 128,
 131,
 156,
 102,
 140,
 138,
 128,
 177,
 112,
 139,
 151,
 152,
 117,
 129,
 133,
 143,
 115,
 126,
 113,
 148,
 163,
 117,
 124,
 99,
 161,
 116,
 130,
 160,
 130,
 118,
 117,
 102,
 115,
 109,
 126,
 96,
 130,
 231,
 215,
 102,
 120,
 146,
 127,
 145,
 144,
 170,
 150,
 119,
 123,
 119,
 102,
 107,
 142,
 101,
 95,
 186,
 133,
 154,
 130,
 132,
 134,
 124,
 128,
 150,
 95,
 153,
 139,
 122,
 127,
 149,
 105,
 149,
 119,
 155,
 88,
 104,
 98,
 105,
 147,
 143,
 165,
 151,
 150,
 116,
 196,
 137,
 104,
 145,
 145,
 106,
 177,
 136,
 145,
 126,
 127,
 138,
 189,
 109,
 119,
 130,
 209,
 124,
 147,
 118,
 137,
 163,
 125,
 113,


In [37]:
len(cls_indices_list)

4132

In [38]:
class_tokens_indices_list = [x - 2 for x in cls_indices_list]

In [39]:
class_tokens_indices_list

[143,
 169,
 218,
 148,
 99,
 128,
 150,
 286,
 112,
 141,
 111,
 186,
 137,
 147,
 122,
 147,
 117,
 127,
 174,
 140,
 133,
 103,
 138,
 177,
 149,
 129,
 117,
 206,
 139,
 123,
 129,
 219,
 205,
 159,
 151,
 134,
 117,
 122,
 155,
 125,
 134,
 196,
 178,
 168,
 146,
 168,
 141,
 168,
 163,
 125,
 102,
 90,
 109,
 131,
 153,
 107,
 129,
 98,
 122,
 96,
 126,
 129,
 154,
 100,
 138,
 136,
 126,
 175,
 110,
 137,
 149,
 150,
 115,
 127,
 131,
 141,
 113,
 124,
 111,
 146,
 161,
 115,
 122,
 97,
 159,
 114,
 128,
 158,
 128,
 116,
 115,
 100,
 113,
 107,
 124,
 94,
 128,
 229,
 213,
 100,
 118,
 144,
 125,
 143,
 142,
 168,
 148,
 117,
 121,
 117,
 100,
 105,
 140,
 99,
 93,
 184,
 131,
 152,
 128,
 130,
 132,
 122,
 126,
 148,
 93,
 151,
 137,
 120,
 125,
 147,
 103,
 147,
 117,
 153,
 86,
 102,
 96,
 103,
 145,
 141,
 163,
 149,
 148,
 114,
 194,
 135,
 102,
 143,
 143,
 104,
 175,
 134,
 143,
 124,
 125,
 136,
 187,
 107,
 117,
 128,
 207,
 122,
 145,
 116,
 135,
 161,
 123,
 111,
 15

In [40]:
# inputs_pe['input_ids'][0][143].item() # correct

In [41]:
# now get a list of input ids at all these indics: they should be either claim = 4366, premise = 18458

In [42]:
list_index_minus_2 = []

for idx, val in enumerate(class_tokens_indices_list):
    at_idx_minus_2 = inputs_train['input_ids'][idx][val].item()
    list_index_minus_2.append(at_idx_minus_2)

In [43]:
set(list_index_minus_2)

{4366, 18458}

In [44]:
# correct

In [45]:
# now make the class token indices 103

In [46]:
for idx, val in enumerate(class_tokens_indices_list):
    inputs_train['input_ids'][idx][val] = 103

In [47]:
# now check if they are 103

In [48]:
unsontroi_check_list = []

for idx, val in enumerate(class_tokens_indices_list):
    at_idx_minus_2 = inputs_train['input_ids'][idx][val].item()
    unsontroi_check_list.append(at_idx_minus_2)

In [49]:
set(unsontroi_check_list)

{103}

In [50]:
# correct

In [51]:
class PeMiniDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [52]:
train_dataset = PeMiniDataset(inputs_train)

In [53]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

### test dataset

In [54]:
df_test = df_sans_mc[df_sans_mc.split == 'TEST']

In [55]:
df_test = df_test.reset_index(drop=True)

In [56]:
df_test

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,nr_preceeding_comps_in_para,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_1
0,essay004,T3,Claim 179 239,the tourism bring large profit for the destina...,Claim,Claim,[],Attack,NotLinked,TEST,...,1,1,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,0.25,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, VERB, ADJ, NOU...",Topic: International tourism is now more commo...,Component: Topic: International tourism is now...
1,essay004,T4,Claim 953 1031,international tourism can create negative impa...,Claim,Claim,[],Support,Linked,TEST,...,7,0,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,0.50,0,Topic: International tourism is now more commo...,"Part Of Speech tags: ADJ, NOUN, VERB, VERB, AD...",Topic: International tourism is now more commo...,Component: Topic: International tourism is now...
2,essay004,T5,Claim 1578 1624,tourism has threatened the nature environments,Claim,Claim,[],Support,Linked,TEST,...,7,0,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,0.75,0,Topic: International tourism is now more commo...,"Part Of Speech tags: NOUN, AUX, VERB, DET, NOU...",Topic: International tourism is now more commo...,Component: Topic: International tourism is now...
3,essay004,T6,Premise 417 530,tourists from different cultures will probably...,Premise,Premise,[],Support,NotLinked,TEST,...,4,3,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,0.50,0,Topic: International tourism is now more commo...,"Part Of Speech tags: NOUN, ADP, ADJ, NOUN, VER...",Topic: International tourism is now more commo...,Component: Topic: International tourism is now...
4,essay004,T7,Premise 532 818,"Take Thailand for example, in the Vietnam War,...",Premise,Premise,[],Support,NotLinked,TEST,...,5,2,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,0.50,0,Topic: International tourism is now more commo...,"Part Of Speech tags: VERB, PROPN, ADP, NOUN, P...",Topic: International tourism is now more commo...,Component: Topic: International tourism is now...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102,essay398,T12,Claim 1484 1589,universities should encourage more girls to ch...,Claim,Claim,[],Support,Linked,TEST,...,0,2,Topic: We can not forcedly put the same number...,Topic: We can not forcedly put the same number...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: NOUN, VERB, VERB, ADJ, NO...",Topic: We can not forcedly put the same number...,Component: Topic: We can not forcedly put the ...
1103,essay398,T13,Premise 1595 1648,this could avoid imbalance of gender in some s...,Premise,Premise,[],Support,NotLinked,TEST,...,1,1,Topic: We can not forcedly put the same number...,Topic: We can not forcedly put the same number...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: DET, VERB, VERB, NOUN, AD...",Topic: We can not forcedly put the same number...,Component: Topic: We can not forcedly put the ...
1104,essay398,T14,Premise 1650 1734,It would affect students' mental health to stu...,Premise,Premise,[],Support,NotLinked,TEST,...,2,0,Topic: We can not forcedly put the same number...,Topic: We can not forcedly put the same number...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: PRON, VERB, VERB, NOUN, P...",Topic: We can not forcedly put the same number...,Component: Topic: We can not forcedly put the ...
1105,essay398,T15,Pr

In [57]:
df_test.split.value_counts()

TEST    1107
Name: split, dtype: int64

In [58]:
prompted_texts_test_l = df_test['prompted_representation_1'][:].tolist()

In [59]:
len(prompted_texts_test_l)

1107

In [60]:
inputs_test = tokenizer(prompted_texts_test_l, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [61]:
inputs_test

{'input_ids': tensor([[ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        ...,
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [62]:
inputs_test['input_ids'].shape

torch.Size([1107, 512])

In [63]:
# for test we do not need labels

In [64]:
#inputs_train['labels'] = inputs_train.input_ids.detach().clone()

In [65]:
#inputs_train

In [66]:
#inputs_train['labels'].shape

In [67]:
# find where the last 102 is

In [68]:
#(inputs_train['input_ids'][0] == 102).nonzero(as_tuple=True)[0].item()

In [69]:
#tokenizer.decode(4366)

In [70]:
#tokenizer.decode(18458)

In [71]:
# claim = 4366, premise = 18458

In [72]:
#(inputs_train['input_ids'][1] == 102).nonzero(as_tuple=True)[0].item()

In [73]:
#inputs_train['input_ids'][1][169]

In [74]:
# confirm that all at [len-2] indices are either claim or premise

In [75]:
cls_indices_list_test = []

for i in range(len(prompted_texts_test_l)):
    cls_idx = (inputs_test['input_ids'][i] == 102).nonzero(as_tuple=True)[0].item()
    cls_indices_list_test.append(cls_idx)

In [76]:
cls_indices_list_test

[129,
 112,
 99,
 138,
 254,
 145,
 123,
 231,
 150,
 154,
 137,
 140,
 200,
 147,
 131,
 106,
 113,
 114,
 117,
 128,
 134,
 138,
 105,
 134,
 106,
 119,
 122,
 108,
 144,
 135,
 103,
 138,
 101,
 127,
 127,
 111,
 115,
 94,
 102,
 118,
 113,
 88,
 98,
 161,
 123,
 144,
 104,
 136,
 151,
 141,
 103,
 115,
 148,
 127,
 158,
 138,
 191,
 174,
 220,
 165,
 165,
 119,
 139,
 112,
 128,
 143,
 110,
 132,
 138,
 122,
 147,
 155,
 145,
 131,
 109,
 155,
 102,
 99,
 106,
 119,
 129,
 119,
 156,
 132,
 146,
 116,
 124,
 106,
 182,
 95,
 171,
 142,
 138,
 116,
 226,
 135,
 147,
 143,
 108,
 130,
 182,
 147,
 123,
 130,
 160,
 188,
 129,
 181,
 97,
 97,
 111,
 157,
 125,
 135,
 163,
 113,
 156,
 110,
 129,
 94,
 112,
 131,
 107,
 137,
 160,
 196,
 114,
 113,
 119,
 142,
 180,
 129,
 128,
 132,
 151,
 107,
 147,
 158,
 207,
 138,
 115,
 139,
 115,
 145,
 137,
 135,
 128,
 191,
 136,
 115,
 115,
 135,
 129,
 145,
 124,
 102,
 186,
 172,
 116,
 234,
 192,
 124,
 136,
 144,
 152,
 187,
 144,
 147,
 

In [77]:
len(cls_indices_list_test)

1107

In [78]:
class_tokens_indices_test_list = [x - 2 for x in cls_indices_list_test]

In [79]:
class_tokens_indices_test_list

[127,
 110,
 97,
 136,
 252,
 143,
 121,
 229,
 148,
 152,
 135,
 138,
 198,
 145,
 129,
 104,
 111,
 112,
 115,
 126,
 132,
 136,
 103,
 132,
 104,
 117,
 120,
 106,
 142,
 133,
 101,
 136,
 99,
 125,
 125,
 109,
 113,
 92,
 100,
 116,
 111,
 86,
 96,
 159,
 121,
 142,
 102,
 134,
 149,
 139,
 101,
 113,
 146,
 125,
 156,
 136,
 189,
 172,
 218,
 163,
 163,
 117,
 137,
 110,
 126,
 141,
 108,
 130,
 136,
 120,
 145,
 153,
 143,
 129,
 107,
 153,
 100,
 97,
 104,
 117,
 127,
 117,
 154,
 130,
 144,
 114,
 122,
 104,
 180,
 93,
 169,
 140,
 136,
 114,
 224,
 133,
 145,
 141,
 106,
 128,
 180,
 145,
 121,
 128,
 158,
 186,
 127,
 179,
 95,
 95,
 109,
 155,
 123,
 133,
 161,
 111,
 154,
 108,
 127,
 92,
 110,
 129,
 105,
 135,
 158,
 194,
 112,
 111,
 117,
 140,
 178,
 127,
 126,
 130,
 149,
 105,
 145,
 156,
 205,
 136,
 113,
 137,
 113,
 143,
 135,
 133,
 126,
 189,
 134,
 113,
 113,
 133,
 127,
 143,
 122,
 100,
 184,
 170,
 114,
 232,
 190,
 122,
 134,
 142,
 150,
 185,
 142,
 145,
 1

In [80]:
# inputs_pe['input_ids'][0][143].item() # correct

In [81]:
# now get a list of input ids at all these indices: they should be either claim = 4366, premise = 18458

In [82]:
list_index_minus_2_test = []

for idx, val in enumerate(class_tokens_indices_test_list):
    at_idx_minus_2 = inputs_test['input_ids'][idx][val].item()
    list_index_minus_2_test.append(at_idx_minus_2)

In [83]:
set(list_index_minus_2)

{4366, 18458}

In [84]:
# correct

In [85]:
# now make the class token indices 103

In [86]:
for idx, val in enumerate(class_tokens_indices_test_list):
    inputs_test['input_ids'][idx][val] = 103

In [87]:
# now check if they are 103

In [88]:
unsontroi_check_list = []

for idx, val in enumerate(class_tokens_indices_test_list):
    at_idx_minus_2 = inputs_test['input_ids'][idx][val].item()
    unsontroi_check_list.append(at_idx_minus_2)

In [89]:
set(unsontroi_check_list)

{103}

In [90]:
# correct

In [91]:
class PeMiniDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [92]:
test_dataset = PeMiniDataset(inputs_test)

In [93]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=True)

## get the model

In [94]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [95]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [96]:
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [97]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir = 'out',
    per_device_train_batch_size=16,
    num_train_epochs=2
)

In [98]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset
)

In [99]:
trainer.train()

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4132
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 518
/tmp/ipykernel_1023/3831771474.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
500,0.156500


Saving model checkpoint to out/checkpoint-500
Configuration saved in out/checkpoint-500/config.json
Model weights saved in out/checkpoint-500/pytorch_model.bin
/tmp/ipykernel_1023/3831771474.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=518, training_loss=0.15111377048144353, metrics={'train_runtime': 524.5492, 'train_samples_per_second': 15.754, 'train_steps_per_second': 0.988, 'total_flos': 2175124572979200.0, 'train_loss': 0.15111377048144353, 'epoch': 2.0})

In [100]:
torch.save(model, 'mask_pe_model_trained')

In [101]:
trainer.save_model("pe_mask_model")

Saving model checkpoint to pe_mask_model
Configuration saved in pe_mask_model/config.json
Model weights saved in pe_mask_model/pytorch_model.bin


### inferences

In [100]:
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [106]:
# for idx, batch in enumerate(test_loader):
#     print(idx, len(batch['input_ids']))

0 16
1 16
2 16
3 16
4 16
5 16
6 16
7 16
8 16
9 16
10 16
11 16
12 16
13 16
14 16
15 16
16 16
17 16
18 16
19 16
20 16
21 16
22 16
23 16
24 16
25 16
26 16
27 16
28 16
29 16
30 16
31 16
32 16
33 16
34 16
35 16
36 16
37 16
38 16
39 16
40 16
41 16
42 16
43 16
44 16
45 16
46 16
47 16
48 16
49 16
50 16
51 16
52 16
53 16
54 16
55 16
56 16
57 16
58 16
59 16
60 16
61 16
62 16
63 16
64 16
65 16
66 16
67 16
68 16
69 3


/tmp/ipykernel_101/3831771474.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [107]:
# for b in test_loader:
#     break

/tmp/ipykernel_101/3831771474.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [109]:
# b['input_ids'].shape

torch.Size([16, 512])

In [101]:
torch.cuda.empty_cache()

In [116]:
#device_new = torch.device('cpu')

In [117]:
#model.to(device_new)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [102]:
trainer.model.device

device(type='cuda', index=0)

In [127]:
#tr_tst_dl = trainer.get_test_dataloader(test_dataset)

In [129]:
# for b in tr_tst_dl:
#     break

/tmp/ipykernel_101/3831771474.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [143]:
#b

{'input_ids': tensor([[ 101, 6922, 1024,  ...,    0,    0,    0],
         [ 101, 6922, 1024,  ...,    0,    0,    0],
         [ 101, 6922, 1024,  ...,    0,    0,    0],
         ...,
         [ 101, 6922, 1024,  ...,    0,    0,    0],
         [ 101, 6922, 1024,  ...,    0,    0,    0],
         [ 101, 6922, 1024,  ...,    0,    0,    0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [119]:
#test_dataset[0]

/tmp/ipykernel_101/3831771474.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'input_ids': tensor([  101,  6922,  1024,  8476,  1024,  2248,  6813,  2003,  2085,  2062,
          2691,  2084,  2412,  2077,  1010,  6251,  1024,  2096,  2070,  2453,
          2228,  1996,  6813,  3288,  2312,  5618,  2005,  1996,  7688,  3032,
          1010,  1045,  2052, 27481,  2008,  2023,  3068,  2038,  5360,  1996,
          3451, 12332,  1998,  5591,  1996,  3019,  4044,  1997,  1996,  7538,
         14345,  1012,  1010,  2034,  2030,  2197,  1999,  9491,  1024,  2748,
          1010,  2034,  1999, 20423,  1024,  2053,  1010,  2197,  1999, 20423,
          1024,  2053,  1010,  1999,  1999,  4955,  1024,  2748,  1010,  2003,
          1999,  7091,  1024,  2053,  1012,  2112,  1997,  4613, 22073,  1024,
         20010,  1010, 15156,  1010, 12034,  1010,  4748,  3501,  1010, 15156,
          1010,  4748,  2361,  1010, 20010,  1010, 15156,  1010, 15156,  1012,
          2003,  2023,  6922,  1037, 18458,  1010,  1037,  4366,  2030,  1037,
          2350,  4366,  1029,  2023,  6

In [106]:
#test_dataset[0:8]

/tmp/ipykernel_590/3831771474.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'input_ids': tensor([[ 101, 6922, 1024,  ...,    0,    0,    0],
         [ 101, 6922, 1024,  ...,    0,    0,    0],
         [ 101, 6922, 1024,  ...,    0,    0,    0],
         ...,
         [ 101, 6922, 1024,  ...,    0,    0,    0],
         [ 101, 6922, 1024,  ...,    0,    0,    0],
         [ 101, 6922, 1024,  ...,    0,    0,    0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [107]:
from transformers import BatchEncoding

In [123]:
#whole_batch = BatchEncoding(test_dataset[:]).to(device)

/tmp/ipykernel_590/3831771474.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [124]:
#whole_batch['input_ids'].shape

torch.Size([1107, 512])

In [113]:
# below thing is nice.

In [113]:
#first_batch['input_ids'].shape

torch.Size([8, 512])

In [111]:
#test_outputs = trainer.predict(first_batch['input_ids'], first_batch['attention_mask'])

***** Running Prediction *****
  Num examples = 8
  Batch size = 8


ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_ids,attention_mask,token_type_ids,position_ids,head_mask,inputs_embeds,encoder_hidden_states,encoder_attention_mask,labels,output_attentions,output_hidden_states,return_dict,label_ids,labels,label.

In [144]:
#test_outputs = model(b['input_ids'], b['attention_mask'])

In [146]:
#test_outputs['logits'].shape

torch.Size([8, 512, 30522])

In [152]:
# model.to(device_new)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [117]:
#batch_output_try = model(first_batch['input_ids'], first_batch['attention_mask'])

In [120]:
#batch_output_try['logits'].shape

torch.Size([8, 512, 30522])

In [153]:
#batch_predictions = []

In [2]:
# for idx, batch in enumerate(test_loader):
#     batch_outputs = model(batch['input_ids'], batch['attention_mask'])
#     batch_predictions.append(batch_outputs)

NameError: name 'test_loader' is not defined

In [125]:
#full_output = model(whole_batch['input_ids'], whole_batch['attention_mask'])

RuntimeError: CUDA out of memory. Tried to allocate 12.97 GiB (GPU 0; 23.88 GiB total capacity; 18.05 GiB already allocated; 5.01 GiB free; 18.17 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [126]:
### new work

In [128]:
len(test_dataset)

1107

In [181]:
batch_size = 16
nr_batches = len(test_dataset) // batch_size

In [182]:
complete_outputs = []

In [183]:
torch.cuda.empty_cache()

In [184]:
del trainer

NameError: name 'trainer' is not defined

In [185]:
del train_loader

NameError: name 'train_loader' is not defined

In [186]:
device_new = torch.device('cpu')

In [187]:
model.to(device_new)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
torch.cuda.empty_cache()

for i in range(nr_batches + 1):
    print(i)
    
    if i != nr_batches:
        
        batch = BatchEncoding(test_dataset[i*batch_size:(i+1)*batch_size]).to(device_new)
    
    else:
        
        batch = BatchEncoding(test_dataset[i*batch_size:len(test_dataset)]).to(device_new)
        
    output = model(batch['input_ids'], batch['attention_mask'])['logits']
    torch.cuda.empty_cache()
    #del batch
    complete_outputs.append(output)

0


/tmp/ipykernel_590/3831771474.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


1
